In [3]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)


In [5]:
import requests

url="https://ec.europa.eu/esco/api/search"
params={
    "text": "python",
    "language": "en",
    "type": "skill"
}

response = requests.get(url, params=params)

import json
print(json.dumps(response.json(), indent=2))

{
  "total": 2,
  "offset": 0,
  "limit": 20,
  "text": "python",
  "language": "en",
  "type": [
    "skill"
  ],
  "isInScheme": null,
  "facet": null,
  "_links": {
    "self": {
      "href": "https://ec.europa.eu/esco/api/search?text=python&type=skill&limit=20&offset=0&full=false&viewObsolete=false&language=en"
    }
  },
  "_embedded": {
    "results": [
      {
        "className": "Skill",
        "classId": "http://data.europa.eu/esco/model#Skill",
        "uri": "http://data.europa.eu/esco/skill/ccd0a1d9-afda-43d9-b901-96344886e14d",
        "searchHit": "Python (computer programming)",
        "title": "Python (computer programming)",
        "preferredLabel": {
          "de": "Python (Computerprogrammierung)",
          "no": "Python (dataprogrammering)",
          "fi": "Python (tietokoneohjelmointi)",
          "en-us": "Python",
          "pt": "Python (programa\u00e7\u00e3o inform\u00e1tica)",
          "bg": "Python (\u043a\u043e\u043c\u043f\u044e\u0442\u044a\u0440\u0